In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [ ]:
data = ""
with open("/content/drive/MyDrive/AirCode.txt", "r") as f:
  data = f.read()
  f.close()

data = data.split("\n")
airport_codes = {}
for d in data:
  temp = d.split(":")
  airport_codes[temp[0]] = temp[1]

In [ ]:
bird_strike_df = bird_strike_df.replace({"AIRPORT_ID": airport_codes})

In [ ]:
# Convert date formats to match
transport_df['FL_DATE'] = pd.to_datetime(transport_df['FL_DATE'])
bird_strike_df['INCIDENT_DATE '] = pd.to_datetime(bird_strike_df['INCIDENT_DATE '])

# Standardize airport codes
transport_df['FL_NUMBER'] = transport_df['FL_NUMBER'].astype(str).str.strip().str.upper()
bird_strike_df['FLT'] = bird_strike_df['FLT'].str.strip().str.upper()

# Standardize airport codes
transport_df['ORIGIN'] = transport_df['ORIGIN'].str.strip().str.upper()
bird_strike_df['AIRPORT_ID'] = bird_strike_df['AIRPORT_ID'].str.strip().str.upper()

# Merge the datasets using Airport Code and Date as the merge keys
merged_df = pd.merge(transport_df, bird_strike_df,
                     left_on=["ORIGIN", "FL_DATE", "FL_NUMBER"],
                     right_on=["AIRPORT_ID", "INCIDENT_DATE ", "FLT"],
                     how="inner")

# Check if the merged dataset is empty
if merged_df.empty:
    print("No matching records found.")
else:
    # Post-merge processing
    # Handle missing values, create new features, etc.
    merged_df.fillna(0, inplace=True)

    # Save the merged dataset
    merged_df.to_csv("/content/drive/MyDrive/merged_data_full.csv", index=False)

In [ ]:
import os
import zipfile
import pandas as pd

def combine_csv_from_zip_by_year(folder_path, year):
    all_dataframes = []
    zip_files = [f for f in os.listdir(folder_path) if f.endswith('.zip') and str(year) in f]

    print(f"Number of zip files for the year {year}: {len(zip_files)}")

    for zip_filename in zip_files:
        zip_file_path = os.path.join(folder_path, zip_filename)
        with zipfile.ZipFile(zip_file_path, 'r') as z:
            for file_name in z.namelist():
                if file_name.endswith('.csv'):
                    with z.open(file_name) as file:
                        df = pd.read_csv(file)
                        all_dataframes.append(df)

    if all_dataframes:
        combined_df = pd.concat(all_dataframes, ignore_index=True)
        return combined_df
    else:
        return pd.DataFrame()

folder_path = '/content/drive/MyDrive/Transport_5_years'

year_to_combine = 2023
combined_csv = combine_csv_from_zip_by_year(folder_path, year_to_combine)

if not combined_csv.empty:
    output_directory = '/content/drive/MyDrive/Transport_5_years'
    output_file = f'combined_transport_{year_to_combine}.csv'
    output_path = os.path.join(output_directory, output_file)

    os.makedirs(output_directory, exist_ok=True)

    combined_csv.to_csv(output_path, index=False)
    print(f"The combined CSV file for the year {year_to_combine} has been saved to:", output_path)
else:
    print(f"No CSV files found for the year {year_to_combine}.")


Number of zip files for the year 2023: 12
The combined CSV file for the year 2023 has been saved to: /content/drive/MyDrive/Transport_5_years/combined_transport_2023.csv


In [ ]:
data = ""
with open("/content/drive/MyDrive/AirCode.txt", "r") as f:
  data = f.read()
  f.close()

data = data.split("\n")
airport_codes = {}
for d in data:
  temp = d.split(":")
  airport_codes[temp[0]] = temp[1]

airlines = pd.read_csv('/content/drive/MyDrive/airlines.csv')
airports = pd.read_csv('/content/drive/MyDrive/airports.csv')
airports = airports.rename(columns={'IATA_CODE':'ORIGIN'})
airlines = airlines.rename(columns={'IATA_CODE':'OP_UNIQUE_CARRIER'})

In [ ]:
transport_df = pd.read_csv('/content/drive/MyDrive/Transport_5_years/combined_transport_2023.csv')
bird_strike_df = pd.read_excel('/content/drive/MyDrive/bird_5_years/2023.xlsx')

In [ ]:
bird_strike_df = bird_strike_df.replace({"AIRPORT_ID": airport_codes})
transport_df = transport_df.merge(airports, on='ORIGIN')
transport_df = transport_df.merge(airlines, on='OP_UNIQUE_CARRIER')

In [ ]:
transport_df['FL_DATE'] = pd.to_datetime(transport_df['FL_DATE'])
bird_strike_df['INCIDENT_DATE '] = pd.to_datetime(bird_strike_df['INCIDENT_DATE '])

transport_df['OP_CARRIER_FL_NUM'] = transport_df['OP_CARRIER_FL_NUM'].astype(str).str.strip().str.upper()
bird_strike_df['FLT'] = bird_strike_df['FLT'].str.strip().str.upper()

transport_df['ORIGIN'] = transport_df['ORIGIN'].str.strip().str.upper()
bird_strike_df['AIRPORT_ID'] = bird_strike_df['AIRPORT_ID'].str.strip().str.upper()

merged_df = pd.merge(transport_df, bird_strike_df,
                     left_on=["ORIGIN", "FL_DATE", "OP_CARRIER_FL_NUM"],
                     right_on=["AIRPORT_ID", "INCIDENT_DATE ", "FLT"],
                     how="inner")

if merged_df.empty:
    print("No matching records found.")
else:
    merged_df.fillna(0, inplace=True)

    merged_df.to_csv("/content/drive/MyDrive/merged_2023.csv", index=False)

<ipython-input-22-a27de5794629>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  transport_df['FL_DATE'] = pd.to_datetime(transport_df['FL_DATE'])


In [ ]:
merged_2023_df = pd.read_csv('/content/drive/MyDrive/merged_2023.csv')

In [ ]:
print(len(merged_2023_df))

3066


In [ ]:
import pandas as pd

file_paths = [
    '/content/drive/MyDrive/merged_2019.csv',
    '/content/drive/MyDrive/merged_2020.csv',
    '/content/drive/MyDrive/merged_2021.csv',
    '/content/drive/MyDrive/merged_2022.csv',
    '/content/drive/MyDrive/merged_2023.csv'
]

dataframes = [pd.read_csv(file) for file in file_paths]

combined_csv = pd.concat(dataframes, ignore_index=True)

output_path = '/content/drive/MyDrive/combined_all'

combined_csv.to_csv(output_path, index=False)

print(f"The combined CSV file has been saved to: {output_path}")


The combined CSV file has been saved to: /content/drive/MyDrive/combined_all


In [3]:
combined_df = pd.read_csv('/content/drive/MyDrive/data/combined_all')

<ipython-input-3-2f5ee09acf9d>:1: DtypeWarning: Columns (60,134) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv('/content/drive/MyDrive/data/combined_all')


In [4]:
print(len(combined_df))

10987


In [7]:
print(combined_df.columns)

Index(['YEAR', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE',
       'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID', 'TAIL_NUM',
       'OP_CARRIER_FL_NUM', 'ORIGIN_AIRPORT_ID',
       ...
       'NR_INJURIES', 'NR_FATALITIES', 'COMMENTS', 'REPORTER_NAME',
       'REPORTER_TITLE', 'SOURCE', 'PERSON', 'LUPDATE', 'IMAGE', 'TRANSFER'],
      dtype='object', length=149)


In [5]:
print(combined_df.head)

<bound method NDFrame.head of        YEAR  MONTH  DAY_OF_MONTH  DAY_OF_WEEK     FL_DATE OP_UNIQUE_CARRIER  \
0      2019     10             2            3  2019-10-02                AA   
1      2019     10             2            3  2019-10-02                DL   
2      2019     10             5            6  2019-10-05                AA   
3      2019     10             5            6  2019-10-05                DL   
4      2019     10             7            1  2019-10-07                AA   
...     ...    ...           ...          ...         ...               ...   
10982  2023      1             7            6  2023-01-07                HA   
10983  2023      1             7            6  2023-01-07                HA   
10984  2023      7             5            3  2023-07-05                HA   
10985  2023      4             5            3  2023-04-05                HA   
10986  2023      9            21            4  2023-09-21                HA   

       OP_CARRIER_AIR